# Optimize Initial Conditions
## Yabox

In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

# Initialize Ray

In [2]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",1)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.shutdown()
ray.init(object_store_memory=1*GB,memory=1*GB,
         #lru_evict=True,
         driver_object_store_memory=500*MB,num_gpus=4,num_cpus=10,
         ignore_reinit_error=True) # , include_webui=False, ignore_reinit_error=True)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2020-07-18 18:55:22,985	INFO resource_spec.py:212 -- Starting Ray with 0.98 GiB memory available for workers and up to 1.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-18 18:55:23,449	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)

# Load New and Process Data from website data.brasil.io

In [3]:
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

Baixando arquivos brasil.io...
(pid=5665) ray.get_gpu_ids(): 3
(pid=5665) CUDA_VISIBLE_DEVICES: 3


# Functions to Load Processed Data

In [4]:
def load_confirmed(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

def load_dead(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

# Load solver

In [5]:
LoadBH=True
if LoadBH:
    import LearnerICRayNoLoadBH as L 
else:
    import LearnerICRayNoLoad as L 

# Data for States

In [6]:
dfparam = pd.read_csv("data/param_optimized_Yabox_HistMin.csv")
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-16,200,9133723,0,0,714,300,74,3,0.15,0.4385,0.0048
1,ES,2020-04-09,200,553381,0,0,55,250,79,10,0.15,0.4462,0.0192
2,MG,2020-04-01,200,1223239,0,0,224,250,5,6,0.15,0.3014,0.0098
3,RJ,2020-03-14,200,797669,0,0,830,250,20,32,0.15,0.6667,0.0318
4,CE,2020-03-19,200,674642,0,0,740,250,151,16,0.15,0.5523,0.0230
5,PE,2020-03-20,200,825113,0,0,667,250,133,97,0.15,0.4353,0.0187
6,AM,2020-03-24,200,793195,0,0,737,250,48,22,0.15,0.5826,0.0033
7,PA,2020-03-26,200,780715,0,0,903,250,165,79,0.15,0.5310,0.0342
8,PI,2020-03-25,200,825140,0,0,941,250,28,69,0.15,0.3187,0.0289
9,RR,2020-03-20,200,808620,0,0,769,250,42,4,0.15,0.3927,0.0399


In [7]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-07-17,AP,TOTAL,state,33436,499,119,True,845731.0,16.0,3953.50295,0.0149
1,1,2020-07-16,AP,TOTAL,state,33004,493,118,False,845731.0,16.0,3902.42287,0.0149
2,2,2020-07-15,AP,TOTAL,state,32408,488,117,False,845731.0,16.0,3831.95129,0.0151
3,3,2020-07-14,AP,TOTAL,state,31885,483,116,False,845731.0,16.0,3770.11130,0.0151
4,4,2020-07-13,AP,TOTAL,state,31552,478,115,False,845731.0,16.0,3730.73708,0.0151
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353,323304,2020-02-29,SP,TOTAL,state,2,0,5,False,45919049.0,35.0,0.00436,0.0000
3354,323305,2020-02-28,SP,TOTAL,state,2,0,4,False,45919049.0,35.0,0.00436,0.0000
3355,323306,2020-02-27,SP,TOTAL,state,1,0,3,False,45919049.0,35.0,0.00218,0.0000
3356,323307,2020-02-26,SP,TOTAL,state,1,0,2,False,45919049.0,35.0,0.00218,0.0000


# Functions for Optimization

In [8]:
from scipy.integrate import odeint
import sys
import io
import gc

def create_f(state,e0,a0,r0,date, ratio, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(state,date)
        data = load_confirmed(state,date)*(1-ratio)-dead
        cleanRecovered=False
        s0, deltaDate, i0, d0, startNCases, weigthCases, weigthRecov = point
        end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=deltaDate)
        f=L.Learner.remote(state, end_date.strftime("%Y-%m-%d"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                           cleanRecovered, version, data, dead, savedata=False)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead, point,f         

        gc.collect()

        return result
    return fobjective

In [9]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=1)
def opt(state,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, ratio, predict_range, version):

    bounds=[(s0*0.8,s0*1.5),(-2,2),(i0/2,i0*2), (0,d0*1.2), (0,startNCases*1.2),\
              (0.3,0.8),(0.02,0.08)]
    maxiterations=500
    f=create_f(state,e0,a0,r0,date, ratio, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    del f, bounds, data, dead,best_params
    
    return p

# Main Code

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [11]:
states=dfparam.state
display(states)
allStates=True
version="105"
gc.enable()

optimal=[]
if allStates:
    for state in states:
        #remove previous history file
        strFile='./results/history_'+state+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, ratio, predict_range, version))        
else:
    state = "ES" 
    #remove previous history file
    strFile='./results/history_'+state+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    start_date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,wcasesdate,startNCases, ratio, predict_range, version))        

0     SP
1     ES
2     MG
3     RJ
4     CE
5     PE
6     AM
7     PA
8     PI
9     RR
10    AP
Name: state, dtype: object

In [ ]:
optimal=ray.get(optimal)


2020-07-18 18:55:36,811	WARNING worker.py:1090 -- WARNING: 30 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=5829) basinhopping step 0: f 1.47627e+09
(pid=5832) basinhopping step 0: f 2.7137e+08
(pid=5832) basinhopping step 1: f 2.68011e+08 trial_f 2.68011e+08 accepted 1  lowest_f 2.68011e+08
(pid=5832) found new global minimum on step 1 with function value 2.68011e+08
(pid=5934) basinhopping step 0: f 8.7781e+07
(pid=5836) warning: basinhopping: local minimization failure
(pid=5836) basinhopping step 0: f 2.02186e+07
(pid=5829) basinhopping step 1: f 1.46607e+09 trial_f 1.46607e+09 accepted 1  lowest_f 1.46607e+09
(pid=5829) found new global minimum on step 1 with function value 1.46607e+09
(pid=5832) basinhopping step 2: f 1.77724e+08 trial_f 1.77724e+08 accepted 1  lowest_f 1.77724e+08
(pid=5832) found new global minimum on step 2 with function value 1.77724e+08
(pid=5833) basinhopping step 0: f -3.07893e+11
(pid=5828) basinhopping step 0: f -3.89322e+13
(pid=5831) basinhopping step 0: f -5.35304e+11
(pid=5829) basinhopping step 2: f 1.46607e+09 trial_f 1.87557e+09 accepted 0  lowest_

(pid=5830) basinhopping step 4: f -8.18251e+11 trial_f 5.52266e+08 accepted 0  lowest_f -8.18251e+11
(pid=5827) basinhopping step 7: f -1.12213e+11 trial_f 4.48749e+08 accepted 0  lowest_f -1.12213e+11
(pid=5832) basinhopping step 7: f 1.77724e+08 trial_f 7.55876e+08 accepted 0  lowest_f 1.77724e+08
(pid=5827) basinhopping step 8: f -1.12213e+11 trial_f -2.52117e+08 accepted 0  lowest_f -1.12213e+11
(pid=5835) basinhopping step 4: f -3.7939e+11 trial_f 1.92357e+09 accepted 0  lowest_f -3.7939e+11
(pid=5835) basinhopping step 5: f -3.7939e+11 trial_f 1.89079e+11 accepted 0  lowest_f -3.7939e+11
(pid=5832) warning: basinhopping: local minimization failure
(pid=5832) basinhopping step 8: f 1.77724e+08 trial_f 2.7137e+08 accepted 0  lowest_f 1.77724e+08
(pid=5834) basinhopping step 4: f -7.19078e+11 trial_f -3.45192e+11 accepted 0  lowest_f -7.19078e+11
(pid=5831) basinhopping step 8: f -5.35304e+11 trial_f 9.44599e+09 accepted 0  lowest_f -5.35304e+11
(pid=5829) basinhopping step 7: f 3.2

(pid=5931) basinhopping step 2: f -6.16079e+11 trial_f 5.43681e+11 accepted 0  lowest_f -6.16079e+11
(pid=5888) basinhopping step 1: f -3.07869e+11 trial_f 7.67971e+08 accepted 0  lowest_f -3.07869e+11
(pid=5936) basinhopping step 3: f 3.18571e+07 trial_f 9.58196e+09 accepted 0  lowest_f 3.18571e+07
(pid=5836) basinhopping step 10: f -1.53104e+09 trial_f 2.02186e+07 accepted 0  lowest_f -1.53104e+09
(pid=5884) basinhopping step 5: f 1.05968e+08 trial_f 1.05968e+08 accepted 1  lowest_f 1.05968e+08
(pid=5884) found new global minimum on step 5 with function value 1.05968e+08
(pid=6089) basinhopping step 0: f 3.85454e+07
(pid=5833) basinhopping step 10: f -7.18683e+11 trial_f 4.23714e+10 accepted 0  lowest_f -7.18683e+11
(pid=5931) basinhopping step 3: f -6.16079e+11 trial_f 1.36624e+10 accepted 0  lowest_f -6.16079e+11
(pid=5896) basinhopping step 1: f 1.45676e+08 trial_f 3.88559e+09 accepted 0  lowest_f 1.45676e+08
(pid=5936) basinhopping step 4: f 3.18571e+07 trial_f 5.63172e+08 accept

(pid=5896) basinhopping step 7: f -2.22445e+11 trial_f -2.22445e+11 accepted 1  lowest_f -2.22445e+11
(pid=5896) found new global minimum on step 7 with function value -2.22445e+11
(pid=6115) basinhopping step 0: f 6.73116e+07
(pid=6115) basinhopping step 1: f 6.72515e+07 trial_f 6.72515e+07 accepted 1  lowest_f 6.72515e+07
(pid=6115) found new global minimum on step 1 with function value 6.72515e+07
(pid=5937) basinhopping step 7: f 4.67735e+08 trial_f 1.95904e+09 accepted 0  lowest_f 4.67735e+08
(pid=6115) basinhopping step 2: f 6.72515e+07 trial_f 5.21626e+11 accepted 0  lowest_f 6.72515e+07
(pid=5830) basinhopping step 10: f -1.38826e+12 trial_f -1.38826e+12 accepted 1  lowest_f -1.38826e+12
(pid=5830) found new global minimum on step 10 with function value -1.38826e+12
(pid=5888) basinhopping step 7: f -3.07869e+11 trial_f 6.21932e+08 accepted 0  lowest_f -3.07869e+11
(pid=5931) basinhopping step 8: f -6.16079e+11 trial_f 1.40142e+10 accepted 0  lowest_f -6.16079e+11
(pid=5936) ba

...

(pid=28898) basinhopping step 3: f -3.75057e+11 trial_f 7.99376e+09 accepted 0  lowest_f -3.75057e+11
(pid=28833) basinhopping step 5: f 1.02024e+08 trial_f 1.02024e+08 accepted 1  lowest_f 1.02024e+08
(pid=28833) found new global minimum on step 5 with function value 1.02024e+08
(pid=28794) basinhopping step 8: f -3.18556e+11 trial_f 2.01874e+09 accepted 0  lowest_f -3.18556e+11
(pid=28885) basinhopping step 3: f -4.38291e+11 trial_f 1.35993e+09 accepted 0  lowest_f -4.38291e+11
(pid=28912) basinhopping step 2: f -1.33782e+12 trial_f 7.93547e+08 accepted 0  lowest_f -1.33782e+12
(pid=28937) warning: basinhopping: local minimization failure
(pid=28937) basinhopping step 2: f -2.2123e+11 trial_f 1.53757e+10 accepted 0  lowest_f -2.2123e+11
(pid=28885) basinhopping step 4: f -4.38291e+11 trial_f 9.45543e+09 accepted 0  lowest_f -4.38291e+11
(pid=28912) basinhopping step 3: f -1.33782e+12 trial_f 4.85946e+08 accepted 0  lowest_f -1.33782e+12
(pid=28859) basinhopping step 6: f -8.59587e+11

(pid=29029) basinhopping step 0: f -3.85113e+11
(pid=28911) basinhopping step 9: f 7.55155e+07 trial_f 4.0148e+10 accepted 0  lowest_f 7.55155e+07
(pid=28990) basinhopping step 2: f -1.57608e+13 trial_f 2.93668e+10 accepted 0  lowest_f -1.57608e+13
(pid=28911) basinhopping step 10: f 7.55155e+07 trial_f 8.00876e+09 accepted 0  lowest_f 7.55155e+07
(pid=29030) basinhopping step 0: f 1.30684e+08
(pid=28965) basinhopping step 3: f -1.28652e+11 trial_f 2.99588e+08 accepted 0  lowest_f -1.28652e+11
(pid=28991) basinhopping step 0: f 9.99178e+07
(pid=28990) basinhopping step 3: f -1.57608e+13 trial_f 4.05112e+11 accepted 0  lowest_f -1.57608e+13
(pid=29030) basinhopping step 1: f -5.2747e+11 trial_f -5.2747e+11 accepted 1  lowest_f -5.2747e+11
(pid=29030) found new global minimum on step 1 with function value -5.2747e+11
(pid=29030) basinhopping step 2: f -5.2747e+11 trial_f 3.52332e+11 accepted 0  lowest_f -5.2747e+11
(pid=28938) basinhopping step 5: f -1.74044e+11 trial_f 4.51114e+09 accep

(pid=28964) basinhopping step 7: f -8.70752e+11 trial_f 8.51444e+09 accepted 0  lowest_f -8.70752e+11
(pid=28990) basinhopping step 9: f -1.89123e+13 trial_f 2.49038e+10 accepted 0  lowest_f -1.89123e+13
(pid=29016) basinhopping step 6: f -4.16263e+11 trial_f 3.28307e+11 accepted 0  lowest_f -4.16263e+11
(pid=28912) basinhopping step 10: f -1.33782e+12 trial_f 9.35028e+09 accepted 0  lowest_f -1.33782e+12
(pid=28965) basinhopping step 9: f -1.28652e+11 trial_f 5.41816e+09 accepted 0  lowest_f -1.28652e+11
(pid=28846) basinhopping step 10: f -3.39736e+11 trial_f -3.39736e+11 accepted 1  lowest_f -3.39736e+11
(pid=28846) found new global minimum on step 10 with function value -3.39736e+11
(pid=29029) basinhopping step 7: f -3.85113e+11 trial_f 1.47097e+09 accepted 0  lowest_f -3.85113e+11
(pid=28965) basinhopping step 10: f -1.28652e+11 trial_f 5.27665e+07 accepted 0  lowest_f -1.28652e+11
(pid=29030) basinhopping step 9: f -5.2747e+11 trial_f 5.69751e+10 accepted 0  lowest_f -5.2747e+11

(pid=29056) basinhopping step 7: f -3.37795e+11 trial_f 3.42992e+09 accepted 0  lowest_f -3.37795e+11
(pid=29127) warning: basinhopping: local minimization failure
(pid=29127) basinhopping step 3: f -8.54225e+11 trial_f 5.86819e+08 accepted 0  lowest_f -8.54225e+11
(pid=29056) warning: basinhopping: local minimization failure
(pid=29056) basinhopping step 8: f -3.37795e+11 trial_f 2.11899e+07 accepted 0  lowest_f -3.37795e+11
(pid=29081) basinhopping step 5: f -9.62559e+11 trial_f 2.06977e+10 accepted 0  lowest_f -9.62559e+11
(pid=29081) basinhopping step 6: f -9.62559e+11 trial_f 3.41937e+08 accepted 0  lowest_f -9.62559e+11
(pid=29055) basinhopping step 5: f -4.63465e+11 trial_f 4.69332e+08 accepted 0  lowest_f -4.63465e+11
(pid=29114) basinhopping step 1: f 1.34222e+09 trial_f 5.83103e+09 accepted 0  lowest_f 1.34222e+09
(pid=29114) basinhopping step 2: f 1.33591e+09 trial_f 1.33591e+09 accepted 1  lowest_f 1.33591e+09
(pid=29114) found new global minimum on step 2 with function val

(pid=29111) basinhopping step 5: f -3.26327e+11 trial_f -3.26327e+11 accepted 1  lowest_f -3.26327e+11
(pid=29111) found new global minimum on step 5 with function value -3.26327e+11
(pid=29107) warning: basinhopping: local minimization failure
(pid=29107) basinhopping step 7: f -1.02858e+14 trial_f -6.38556e+13 accepted 0  lowest_f -1.02858e+14
(pid=29081) basinhopping step 9: f -9.62559e+11 trial_f 8.28788e+08 accepted 0  lowest_f -9.62559e+11
(pid=29055) basinhopping step 10: f -4.63465e+11 trial_f 4.67332e+08 accepted 0  lowest_f -4.63465e+11
(pid=29081) basinhopping step 10: f -9.62559e+11 trial_f 3.41937e+08 accepted 0  lowest_f -9.62559e+11
(pid=29107) basinhopping step 8: f -1.02858e+14 trial_f 1.18668e+10 accepted 0  lowest_f -1.02858e+14
(pid=29127) basinhopping step 8: f -8.54225e+11 trial_f 1.64043e+09 accepted 0  lowest_f -8.54225e+11
(pid=29245) basinhopping step 1: f 1.32525e+08 trial_f 2.42749e+08 accepted 0  lowest_f 1.32525e+08
(pid=29230) basinhopping step 5: f -5.25

In [ ]:
for i in range(0,len(states)):    

    state=states[i]
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    